<a href="https://colab.research.google.com/github/nidhimenonn/Thesis/blob/master/68th_round.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from zipfile import ZipFile
!pip install gspread-pandas
from gspread_pandas import Spread, Client

In [2]:
# Mounting the Drive

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
# Installing and Aunthenticating PyDrive

# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

    100% |████████████████████████████████| 993kB 19.6MB/s 


In [4]:
# NUTRITIONAL VALUE PART 1

# link to the dta file
link = 'https://drive.google.com/open?id=1X3kuVEJoq0gYq8wWY27gAA_ASmhDkvjE'

# verify that you have everything after '='
fluff, id = link.split('=')
print (id) 

# store dataset in pandas
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('nss68nutrition.dta')  


1X3kuVEJoq0gYq8wWY27gAA_ASmhDkvjE


In [5]:
# NUTRITIONAL VALUE PART 2

df1 = pd.read_stata('nss68nutrition.dta')
df1.columns




Index(['id_item_68', 'item', 'unit', 'foodgroup', 'rural', 'protein', 'fat',
       'fibre', 'carbohydrate', 'energy_kcal', 'energy_kj', 'calcium', 'iron',
       'betacarotene', 'caroten_total', 'thiamine', 'riboflavin', 'niacin',
       'ascorbic_total', 'ext_source', 'pu_cal', 'pu_prot', 'pu_fat'],
      dtype='object')

In [6]:
# NUTRITIONAL VALUE PART 3

#166, 167, 217,238, 283, 284, 290, 291, 292, 296 --> these are the nutritional values that have been repeated and idk how to work with it so will come back to it later


df2 = df1.reset_index(drop=True).set_index('id_item_68')
df2.index = df2.index.astype(str)

#df2 = df2.loc[~df2.index.duplicated(),:].iloc[:,4:].fillna(0)
df2 = df2.loc[~df2.index.duplicated(),:].fillna(0)


#df2.drop(['166', '167', '217', '238', '283', '284', '290', '291', '292', '296'])

df2.head()


,item,unit,foodgroup,rural,protein,fat,fibre,carbohydrate,energy_kcal,energy_kj,...,betacarotene,caroten_total,thiamine,riboflavin,niacin,ascorbic_total,ext_source,pu_cal,pu_prot,pu_fat
id_item_68,,,,,,,,,,,,,,,,,,,,,
101,rice  PDS,kg,I,0,78.099998,5.500000,37.400002,771.600037,3513.423096,14710.0,...,0.0,469.000000,1.7,0.6,25.100000,0.0,0.0,3460.0,75.0,5.0
102,rice  other sources,kg,I,0,78.099998,5.500000,37.400002,771.600037,3513.423096,14710.0,...,0.0,469.000000,1.7,0.6,25.100000,0.0,0.0,3460.0,75.0,5.0
103,chira,kg,I,0,74.400002,11.400000,34.599998,767.500000,3534.919189,14800.0,...,0.0,336.100006,1.2,0.4,16.000000,0.0,0.0,3460.0,66.0,12.0
104,"khoi, lawa",kg,I,0,74.699997,16.200001,25.599998,776.799988,3534.919189,14800.0,...,0.0,504.599976,1.1,0.4,18.700001,0.0,0.0,3250.0,75.0,1.0
105,muri,kg,I,0,74.699997,16.200001,25.599998,776.799988,3534.919189,14800.0,...,0.0,504.599976,1.1,0.4,18.700001,0.0,0.0,3250.0,75.0,1.0


In [0]:
# NUTRITIONAL VALUE PART 4

# extracting only the items that have the units of kg, l, and g
df3 = (df2.loc[df2['unit'].isin(['kg','litre', 'g'])])

# converting the units 
#kg --> kg, g --> kg (*0.001), l --> kg(*1)
# we want everything to be in kgs because then we can find the differences in nutritional value in kg 
cols = ['protein', 'fat', 'fibre',
       'carbohydrate', 'energy_kcal', 'energy_kj', 'calcium', 'iron',
       'betacarotene', 'caroten_total', 'thiamine', 'riboflavin', 'niacin',
       'ascorbic_total', 'ext_source', 'pu_cal', 'pu_prot', 'pu_fat']

df31 = pd.DataFrame()

for col in cols:
  df31[col] = [df3[col][i]/1000 if df3['unit'][i] =='g' else df3[col][i] for i in df3.index]


In [8]:
# NUTRITIONAL VALUE PART 5

#adding the index values to the new df4
df31.columns.name = "id_item_68"
df31.index = (['101', '102', '103', '104', '105', '106', '107', '108', '110', '111',
       '112', '113', '114', '115', '116', '117', '118', '120', '121', '122',
       '139', '140', '141', '142', '143', '144', '145', '146', '147', '148',
       '150', '151', '152', '160', '161', '162', '163', '164', '165', '171',
       '172', '173', '174', '175', '180', '181', '182', '183', '184', '185',
       '191', '192', '193', '194', '195', '196', '200', '201', '202', '203',
       '204', '205', '206', '207', '208', '210', '211', '212', '213', '214',
       '215', '221', '222', '226', '227', '230', '231', '232', '233', '234',
       '235', '236', '237', '240', '241', '242', '243', '244', '245', '246',
       '247', '275', '276', '321', '322', '323', '324'])
df31.head()

id_item_68,protein,fat,fibre,carbohydrate,energy_kcal,energy_kj,calcium,iron,betacarotene,caroten_total,thiamine,riboflavin,niacin,ascorbic_total,ext_source,pu_cal,pu_prot,pu_fat
101,78.099998,5.500000,37.400002,771.600037,3513.423096,14710.0,81.099998,7.200000,0.0,469.000000,1.7,0.6,25.100000,0.0,0.0,3460.0,75.0,5.0
102,78.099998,5.500000,37.400002,771.600037,3513.423096,14710.0,81.099998,7.200000,0.0,469.000000,1.7,0.6,25.100000,0.0,0.0,3460.0,75.0,5.0
103,74.400002,11.400000,34.599998,767.500000,3534.919189,14800.0,91.899994,44.599998,0.0,336.100006,1.2,0.4,16.000000,0.0,0.0,3460.0,66.0,12.0
104,74.699997,16.200001,25.599998,776.799988,3534.919189,14800.0,150.899994,45.500000,0.0,504.599976,1.1,0.4,18.700001,0.0,0.0,3250.0,75.0,1.0
105,74.699997,16.200001,25.599998,776.799988,3534.919189,14800.0,150.899994,45.500000,0.0,504.599976,1.1,0.4,18.700001,0.0,0.0,3250.0,75.0,1.0


In [0]:
# FOOD EXPENDITURE DATA PART 1 

# extract only the food expenditure file that we need to analyze from a zipped file 

with ZipFile('/content/gdrive/My Drive/NSSOData/Nss68_1.0_Type2_new format.zip', 'r') as zip:
  #for fn in zip.namelist():
    #print(fn)
  zip.extract("Nss68_1.0_Type2_new format/consumption of cereals-pulses- milk and milk products  during the last 30 days   - block 5.1- 5.2- 6 - level 5 - type 2 -  68.dta","./")

In [0]:
# FOOD EXPENDITURE DATA PART 2

# here we list the files within the Zipfile

with ZipFile('/content/gdrive/My Drive/NSSOData/Nss68_1.0_Type2_new format.zip', 'r') as zip:
  for fn in zip.namelist():
    print(fn)

In [0]:
# FOOD EXPENDITURE DATA PART 3

# here we open the consumption dta file in stata format

import pandas as pd

df4 = pd.read_stata("Nss68_1.0_Type2_new format/consumption of cereals-pulses- milk and milk products  during the last 30 days   - block 5.1- 5.2- 6 - level 5 - type 2 -  68.dta")


In [11]:
df4.head()

,Round_Centre_Code,FSU_Serial_No,Round,Sch_no,Sample,Sector,State_region,District,Stratum,Sub_Stratum_No,...,Total_Consumption_Value,Source_Code,nss,nsc,mlt,hhid,State_code,District_Code,Combined_Multiplier,Subsample_multiplier
0,009,71558,68,010,1,1,171,01,01,01,...,420.0,1,2,4,64816,715581201,17,1701,324.08,648.16
1,010,71558,68,010,1,1,171,01,01,01,...,255.0,1,2,4,64816,715581201,17,1701,324.08,648.16
2,011,71558,68,010,1,1,171,01,01,01,...,15.0,1,2,4,64816,715581201,17,1701,324.08,648.16
3,012,71558,68,010,1,1,171,01,01,01,...,60.0,1,2,4,64816,715581201,17,1701,324.08,648.16
4,013,71558,68,010,1,1,171,01,01,01,...,750.0,,2,4,64816,715581201,17,1701,324.08,648.16


In [12]:
# FOOD EXPENDITURE DATA PART 4

# here we extract the data from UP because it has a max difference in Control vs Treatment groups of 3 districts!

df41 = df4.query("State_code=='09'")
df41.columns

#a = df5['District'].unique()

#df42 = df41.filter(['hhid', 'District'])
#df42 = df42.fillna(0)


Index(['Round_Centre_Code', 'FSU_Serial_No', 'Round', 'Sch_no', 'Sample',
       'Sector', 'State_region', 'District', 'Stratum', 'Sub_Stratum_No',
       'Schedule_type', 'Sub_Round', 'Sub_Sample', 'FOD_Sub_region',
       'Hamlet_Group_Sub_Stratum_no', 'Second_Stage_Stratum_No',
       'Sample_hhld_no', 'Level', 'Item_Code', 'Home_Produce_Quantity',
       'Home_Produce_Value', 'Total_Consumption_Quantity',
       'Total_Consumption_Value', 'Source_Code', 'nss', 'nsc', 'mlt', 'hhid',
       'State_code', 'District_Code', 'Combined_Multiplier',
       'Subsample_multiplier'],
      dtype='object')

In [0]:
# let us look at district data here

#dis = df5.query("District == '07'")
#a = dis['hhid'].unique()
#print(a)

In [13]:
UPfood = df41[['hhid','Item_Code','Total_Consumption_Quantity']].set_index(['hhid','Item_Code']).squeeze().unstack('Item_Code')

#once again dropped the values that repeat
UPfood.drop(columns = ['166', '167', '217', '238', '283', '284', '290', '291', '292', '296'])
UPfood = UPfood.fillna(0)
UPfood.columns.name = "id_item_68"
UPfood.head()



#UPfood2 = UPfood.loc['290']


id_item_68,101,102,103,104,105,106,107,108,110,111,...,336,337,338,340,341,342,343,344,345,349
hhid,,,,,,,,,,,,,,,,,,,,,
471001101,0.0,15000.0,0.0,0.0,0.0,0.0,0.0,18000.0,0.0,0.0,...,3.0,0.0,14000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
471001102,0.0,32000.0,0.0,0.0,0.0,0.0,0.0,40000.0,0.0,0.0,...,5.0,0.0,20000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
471001201,0.0,8000.0,0.0,0.0,0.0,0.0,18000.0,0.0,0.0,0.0,...,4.0,0.0,10000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
471001202,0.0,20000.0,0.0,0.0,0.0,0.0,18000.0,0.0,0.0,0.0,...,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
471001203,15000.0,20000.0,0.0,0.0,0.0,0.0,18000.0,20000.0,0.0,0.0,...,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
use = list(set(df31.index.tolist()).intersection(UPfood.columns))
N = UPfood[use].dot(df31.loc[use,:])
N.head()


id_item_68,protein,fat,fibre,carbohydrate,energy_kcal,energy_kj,calcium,iron,betacarotene,caroten_total,thiamine,riboflavin,niacin,ascorbic_total,ext_source,pu_cal,pu_prot,pu_fat
hhid,,,,,,,,,,,,,,,,,,
471001101,5.973560e+06,4.481120e+06,3.506970e+06,3.827571e+07,2.066120e+08,8.650432e+08,6.046860e+07,1.663050e+06,7.762500e+07,3.129451e+08,154989.993036,79010.000142,1.075490e+06,2.217570e+06,0.0,222578000.0,6313900.0,3599600.0
471001102,9.717452e+06,4.427497e+06,6.934697e+06,6.331788e+07,3.283252e+08,1.374632e+09,6.940681e+07,2.617684e+06,2.019553e+08,6.339965e+08,284540.717015,126551.218949,2.109340e+06,5.577841e+06,0.0,347064500.0,10414600.0,4101900.0
471001201,4.057859e+06,2.407071e+06,2.929611e+06,2.569629e+07,1.357822e+08,5.684930e+08,3.501901e+07,1.246130e+06,1.061525e+08,3.376566e+08,122255.358922,55385.609740,8.036173e+05,2.974783e+06,0.0,145929000.0,4360450.0,1970450.0
471001202,4.541320e+06,1.541440e+06,3.352240e+06,3.245772e+07,1.608777e+08,6.735627e+08,1.528130e+07,1.209500e+06,6.988970e+07,2.327145e+08,137679.993957,45819.999284,1.084080e+06,1.796040e+06,0.0,165292000.0,4732800.0,1085200.0
471001203,8.399825e+06,2.124050e+06,6.531665e+06,5.778154e+07,2.869124e+08,1.201245e+09,3.336226e+07,2.195835e+06,1.625456e+08,5.121965e+08,261814.986855,96754.998875,2.040205e+06,3.740275e+06,0.0,296886000.0,8947600.0,2041950.0


In [56]:
!pip install --upgrade -q gspread
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.open('UP District and Code').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
import pandas as pd
UProws = pd.DataFrame.from_records(rows)


[['23', 'Kheri', '1'], ['24', 'Sitapur', '1'], ['25', 'Hardoi', '1'], ['26', 'Unnao', '1'], ['28', 'Rae Bareli ', '1'], ['35', 'Jalaun', '1'], ['37', 'Lalitpur', '1'], ['38', 'Hamirpur', '1'], ['39', 'Mahoba', '1'], ['40', 'Banda', '1'], ['41', 'Chitrakoot', '1'], ['42', 'Fatehpur', '1'], ['43', 'Pratapgarh', '1'], ['44', 'Kaushambi', '1'], ['46', 'Barabanki', '1'], ['58', 'Gorakhpur', '1'], ['59', 'Kushinagar', '1'], ['61', 'Azamgarh', '1'], ['64', 'Jaunpur', '1'], ['66', 'Chandauli', '1'], ['69', 'Mirzapur', '1'], ['70', 'Sonbhadra', '1'], ['17', 'Etah', '2'], ['19', 'Budaun', '2'], ['29', 'Farrukhabad', '2'], ['33', 'Kanpur Dehat ', '2'], ['36', 'Jhansi', '2'], ['48', 'Ambedkar Nag', '2'], ['49', 'Sultanpur', '2'], ['50', 'Bahraich', '2'], ['51', 'Shrawasti', '2'], ['52', 'Balrampur', '2'], ['53', 'Gonda', '2'], ['54', 'Siddharthnagar', '2'], ['55', 'Basti', '2'], ['56', 'S. Kabir Nagar ', '2'], ['57', 'Maharajganj', '2'], ['62', 'Mau', '2'], ['63', 'Ballia', '2'], ['1', 'Saharanpur

In [60]:
#UProws.rename(columns={"0": "District", "1": "District Name (68)", "2":"MGNREGA Phase"})
UProws.columns = ['District', 'District Name (68)', 'MGNREGA Phase']
UProws.head()

,District,District Name (68),MGNREGA Phase
0,23,Kheri,1
1,24,Sitapur,1
2,25,Hardoi,1
3,26,Unnao,1
4,28,Rae Bareli,1
5,35,Jalaun,1
6,37,Lalitpur,1
7,38,Hamirpur,1
8,39,Mahoba,1
9,40,Banda,1


In [58]:
# hhid, district, nutrition


P = df41.filter(['hhid', 'District'])
Q = N.merge(P, left_on ='hhid', right_on = 'hhid').drop_duplicates()
Q.head() 

R = Q.groupby('District').mean()
R

S = R.merge(UProws, left_on = 'District', right_on = 'District').drop_duplicates()
S.head()





,District,protein,fat,fibre,carbohydrate,energy_kcal,energy_kj,calcium,iron,betacarotene,...,thiamine,riboflavin,niacin,ascorbic_total,ext_source,pu_cal,pu_prot,pu_fat,District Name (68),MGNREGA Phase
0,10,7.965394e+06,5.583364e+06,6.080910e+06,4.402760e+07,2.533841e+08,1.060869e+09,8.626088e+07,2.515433e+06,1.563447e+08,...,241766.976401,130429.289429,1.484788e+06,4.166627e+06,418.750000,2.791915e+08,8.878307e+06,5.782850e+06,G. Buddha Nagar,3
1,11,8.397781e+06,5.130133e+06,6.562990e+06,4.736506e+07,2.653773e+08,1.111081e+09,8.622464e+07,2.652606e+06,1.382284e+08,...,263342.015226,137664.728178,1.598947e+06,3.343071e+06,135.625000,2.914440e+08,9.416951e+06,5.423828e+06,Bulandshahr,3
2,12,9.131631e+06,5.425430e+06,7.320498e+06,5.171891e+07,2.887124e+08,1.208781e+09,8.658040e+07,2.945124e+06,1.613670e+08,...,291607.385905,146742.336023,1.769086e+06,3.600045e+06,155.937500,3.156692e+08,1.022004e+07,5.658942e+06,Aligarh,3
3,13,9.324041e+06,5.883640e+06,7.282500e+06,5.160329e+07,2.937371e+08,1.229819e+09,9.698486e+07,2.904025e+06,1.621269e+08,...,293914.434358,155567.101491,1.770205e+06,3.693676e+06,138.541667,3.229460e+08,1.049744e+07,6.405424e+06,Hathras,3
4,14,7.805320e+06,4.905786e+06,6.262003e+06,4.396920e+07,2.478592e+08,1.037737e+09,8.168436e+07,2.533213e+06,1.660904e+08,...,251581.393377,132207.524803,1.485691e+06,3.432313e+06,26.562500,2.728738e+08,8.786010e+06,5.219341e+06,Mathura,3


In [0]:
T = S.groupby('MGNREGA Phase').mean()
U = S.groupby('MGNREGA Phase').std()
V = S.groupby('MGNREGA Phase').count()


T.to_pickle('68 mean')


In [0]:
|# Joining dataframes
# df6 = UPfood.dot(df2)
# df.join(other.set_index('key'), on='key')
# df8 = UPfood.join(df2, how='outer')
# df8.loc[166]
# UPfood.dot(df2)
# df2.loc[166]
# df2.head()
# df2.index.values


vals = set([int(i) for i in UPfood.index.values])
df2vals = set(df2.index.values)

for num in list(vals.difference(df2vals)):
#  df2.iloc[num] = [0] *  len(df2.columns)
   df2.loc[num] = [0]*len(df2.columns)


len(df2)

In [0]:
# NUTRITIONAL VALUE PART 4

# extracting only the items that have the units of kg, l, and g
#df3 = (df2.loc[df2['unit'].isin(['kg','litre', 'g'])])

# converting the units 
#kg --> kg, g --> kg (*0.001), l --> kg(*1)
# we want everything to be in kgs because then we can find the differences in nutritional value in kg 

#if df3.iloc[df3['unit'].isin(['g'])]:
#  print df3.iloc[df]

#df3 = df3.loc[~df3.index.duplicated(),:].iloc[:,4:].fillna(0)
#print(df3)
#df3.shape

In [0]:

df8 = set(df2.index.values) - set(UPfood.index.values)
len(df8)


#len(df2vals)


#df2 = df2.T
#df6 = UPfood.dot(df2)


In [0]:
# MEASURING WELFARE USING CPI
# pg 184 from pdf


UPfood = df5[['hhid','Item_Code','Total_Consumption_Quantity']].set_index(['hhid','Item_Code']).squeeze().unstack('Item_Code')
UPfood.head()

In [0]:
!pip install --upgrade -q gspread

from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.open('2011 monthly CPI India').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
import pandas as pd
df11 = pd.DataFrame.from_records(rows)

print(df11)

In [0]:
list(df11.columns)
